## Multi-Source and Multi-Model AI Inference
This notebook is an example how to perform AI inferences of multiple models processing multiple video streams.
Each video stream is fed to every model. Each model processes frames from every video stream in multiplexing manner.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, 
located in the same directory as this notebook.

The script may use a web camera(s) or local camera(s) connected to the machine running this code.
The camera index or URL needs to be specified either in the code below by assigning `camera_id` or 
in [env.ini](env.ini) file by defining `CAMERA_ID` variable and assigning `camera_id = None`.

#### Specify video sources and AI model names here

In [ ]:
# list of video sources: it can be video file names, indexes or local cameras or IP camera URLs
video_sources = [
    "Images/Traffic.mp4",
    "Images/TrafficHD.mp4",
]

# list of AI models to use for inferences
# NOTE: they should have the same input size
model_names = [
    "yolo_v5s_hand_det--512x512_quant_n2x_orca_1",
    "yolo_v5s_face_det--512x512_quant_n2x_orca_1",
    "yolo_v5n_car_det--512x512_quant_n2x_orca_1",
    "yolo_v5s_person_det--512x512_quant_n2x_orca_1",
]

# when True, we drop video frames in case when AI performance is not enough to work in real time
# when False, we buffer video frames to keep up with AI performance
allow_frame_drop = True


#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = mytools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, mytools.get_cloud_zoo_url(), mytools.get_token())

In [ ]:
import mystreams

c = mystreams.Composition()

batch_size = len(
    video_sources
)  # set AI server batch size equal to the # of video sources for lowest latency

# create PySDK AI model objects
models = []
for mi, model_name in enumerate(model_names):
    model = zoo.load_model(model_name)
    model.input_image_format = "JPEG"
    model.measure_time = True
    model.eager_batch_size = batch_size
    model.frame_queue_depth = batch_size
    models.append(model)

# check that all models have the same input configuration
models_have_same_input = True
for model in models[1:]:
    if (
        type(model._preprocessor) != type(models[0]._preprocessor)
        or model.model_info.InputH != models[0].model_info.InputH
        or model.model_info.InputW != models[0].model_info.InputW
    ):
        models_have_same_input = False

resizers = []

# create video sources and image resizers
# (we use separate resizers to do resize only once per source when possible, to improve performance),
# connect each resizer to corresponding video source
for src in video_sources:
    source = c.add(mystreams.VideoSourceGizmo(src))
    if models_have_same_input:
        resizer = c.add(
            mystreams.AiPreprocessGizmo(
                models[0], stream_depth=2, allow_drop=allow_frame_drop
            )
        )
    else:
        resizer = c.add(mystreams.FanoutGizmo(allow_drop=allow_frame_drop))

    resizer.connect_to(source)  # connect resizer to video source
    resizers.append(resizer)

# create result combiner
combiner = c.add(mystreams.AiResultCombiningGizmo(len(models)))

# create multi-input detector gizmos,
# connect each detector gizmo to every resizer gizmo,
# connect result combiner gizmo to each detector gizmo
for mi, model in enumerate(models):

    # create AI gizmo (aka detector) from the model
    detector = c.add(
        mystreams.AiSimpleGizmo(model, stream_depth=2, inp_cnt=len(video_sources))
    )

    # connect detector gizmo to each resizer gizmo
    for fi, resizer in enumerate(resizers):
        detector.connect_to(resizer, fi)

    # connect result combiner gizmo to detector gizmo
    combiner.connect_to(detector, mi)

# create multi-window video multiplexing display gizmo
# and connect it to combiner gizmo
win_captions = [f"Camera #{i}: {str(src)}" for i, src in enumerate(video_sources)]
display = c.add(
    mystreams.VideoDisplayGizmo(
        win_captions, show_ai_overlay=True, show_fps=True, multiplex=True
    )
)
display.connect_to(combiner)

# start composition
c.start()
